In [12]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
import ipdb
import re
from datasets import load_dataset
import os
from diversity import compression_ratio, homogenization_score, ngram_diversity_score, extract_patterns, get_pos, pos_patterns, token_patterns, self_repetition_score
import json
from collections import Counter
from random import shuffle

tqdm.pandas()

import matplotlib.pyplot as plt
# import mplcursors
import seaborn as sns
%matplotlib inline
sns.set(style='darkgrid', context='notebook', rc={'figure.figsize':(14,10)}, font_scale=2)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('chained_assignment',None)

# Set random seeds for reproducibility on a specific machine
random.seed(1)
np.random.seed(1)
np.random.RandomState(1)
np.set_printoptions(precision=3)

## Dolly human written responses diversity analysis

In [2]:
with open('../data/sample_personas.txt', 'r') as f:
    personas = [x.strip() for x in f.readlines()]
personas[2]

'a Spanish-speaking patient with severe myopia interested in LASIK eye surgery'

In [3]:
dolly = load_dataset("databricks/databricks-dolly-15k")["train"].filter(lambda row: row['category']=='creative_writing').to_pandas()
sample = pd.read_csv('../data/dolly_creative_prompts_sample.tsv', sep='\t')

In [4]:
sample.head()

,index,instruction,num_tokens,num_tokens_round,num_words_round
0,525,Please propose an argument to convince my mother that she increases the amount of money that I get every month during my studies. I think the current amount is too low.,211,210,180
1,172,"Write a paragraph to refute a claim by a colleague that ancient structures such as Stonehenge, the Great Pyramid are evidence of UFO activities on Earth",189,190,160
2,109,What is the best way to answer an interview question?,128,130,100
3,37,Write the first paragraph of an advertising brochure describing an hotel nearby the Annecy Lake in France,89,90,60
4,295,"Write an intro to a meetup about music, medicine, and machines",313,310,220


In [5]:
sample['response'] = sample['index'].apply(lambda x: dolly.loc[x, 'response'])
sample['prompt_id'] = [i for i in range(len(sample))]
sample.head()

,index,instruction,num_tokens,num_tokens_round,num_words_round,response,prompt_id
0,525,Please propose an argument to convince my mother that she increases the amount of money that I get every month during my studies. I think the current amount is too low.,211,210,180,"Dear Mum, I would like to talk to you about the monthly money I get for my studies. I am very grateful that you support me financially during my studies. At the same time, I feel that the current amount is a little bit too low in order to sustain myself. I estimated my monthly expenses and it seems like the money is not enough. We can go through the details if you want. I want to ask you if you can increase the amount so that I can cover my expenses. I understand if you cannot do this or have other reasons against this. However, I want you to know that if you do not increase my monthly money, I will be forced to work part-time next to my studies. There is research that shows that students who work part-time have worse grades than people who do not have to work part-time to afford their studies. I would love to dedicate my full time to my studies to get the best grades and learning outcomes possible; thus, I would prefer not to work part-time. What do you think?",0
1,172,"Write a paragraph to refute a claim by a colleague that ancient structures such as Stonehenge, the Great Pyramid are evidence of UFO activities on Earth",189,190,160,"There are several strong arguments against concluding that alien races built the structures like the Stonehenge or the Pyramids. First, there is evidence that these magnificent structures were built by humans. For example, tombs of the pyramid builders were found in the vicinity of the Great Pyramind (with human remains!). The technology to move and assemble the massive stones of the Stonehenge has been shown to be available to humans at that time. Second, given that space is vast the chance of finding one tiny planet among billions is infinitesimal. If alien intelligence had indeed found our one inhabited planet they are unlikely to have just created couple of structures and avoided any further contact with Earth and its inhabitants. In science, Occam’s Razor (“the best explanation is the simplest one”) is a guiding principle and until there is much more evidence of alien visitation it would be inadvisable to jump to a conclusion that aliens built these structures.",1
2,109,What is the best way to answer an interview question?,128,130,100,"The first recommended step is to ask clarifying questions to ensure you understand the question correctly and gather the requirements needed to answer effectively. If it is a mathematical or computer science question, go ahead and work your way through the problem while you verbalize your thought process along the way. If it is a situation question, you should use the STAR method. 1. Situation - set the scene and provide context, 2. Task - describe what was needed to be done and why, 3. Action - explain what action you took, and 4. Result - describe the result of your actions and what you learned from the experience",2
3,37,Write the first paragraph of an advertising brochure describing an hotel nearby the Annecy Lake in France,89,90,60,"This fantastic hotel is ideally located, minutes away from the Annecy Lake, one of the purest in Europe. The water temperature reaches 27 degrees celcius at the peak season, allowing the guests to swim and enjoy several water activities including wake surf and pedalo. Finally, this region is known for being the home of several cheese specialties like ""Raclette"" and ""Tartiflette"" to name a few.",3
4,295,"Write an intro to a meetup about music, medicine, and machines",313,310,220,"Music, Medicine, and Machines\n\nJoin us for an imaginative, authentic, and offbeat networking experience that will advance your knowledge of the technologies that impact the health and well-being of our human condition. To make things interesting (but keep things real), our meetup i

In [6]:
prompts = sample['instruction'].values.tolist()

In [7]:
human_responses = sample['response'].values.tolist()

In [ ]:
sample.num_tokens_round.sort_values().tail(5)

In [ ]:
cr = compression_ratio(human_responses, 'gzip')
nds = ngram_diversity_score(human_responses, 4)
joined_pos, tuples = get_pos(human_responses)
# ngrams_pos = token_patterns(joined_pos, 5, 10)
cr_pos = compression_ratio(joined_pos, 'gzip')
# rouge = homogenization_score(human_responses, 'rougel', verbose=False)
# bleu = homogenization_score(human_responses, 'bleu', verbose=False)
srep = self_repetition_score(human_responses)
# print(f"CR: {np.round(cr,2)}\nNDS: {np.round(nds,2)}\nCR-POS: {np.round(cr_pos,2)}\nHS-RougeL: {np.round(rouge,2)}\nself-bleu: {np.round(bleu,2)}\nSelf-rep: {np.round(srep, 2)}")
print(f"CR: {np.round(cr,2)}\nNDS: {np.round(nds,2)}\nCR-POS: {np.round(cr_pos,2)}\nSelf-rep: {np.round(srep, 2)}")

## Main metric calculation function (can find mean and SD over persona column)

In [8]:
def calc_cr_nds_over_personas(df):
    crs = []
    ndss = []
    crs_pos = []
    sreps = []
    for persona_id in tqdm(df.persona_id.unique()):
        responses = df.loc[df.persona_id==persona_id].drop_duplicates(subset=['prompt_id'])['response'].values.tolist()
        cr = compression_ratio(responses, 'gzip')
        nds = ngram_diversity_score(responses, 4)
        #CR-POS
        joined_pos, tuples = get_pos(responses)
        # ngrams_pos = token_patterns(joined_pos, 5, 10)
        cr_pos = compression_ratio(joined_pos, 'gzip')
        srep = self_repetition_score(responses, verbose=False)
        crs.append(cr)
        ndss.append(nds)
        crs_pos.append(cr_pos)
        sreps.append(srep)
    print(f"CR: {np.round(np.mean(crs),2)} ± {np.round(np.std(crs),2)}\nCR-POS: {np.round(np.mean(crs_pos),2)} ± {np.round(np.std(crs_pos),2)}\nNDS: {np.round(np.mean(ndss),2)} ± {np.round(np.std(ndss),2)}\nSelf-rep:{np.round(np.mean(sreps),2)} ± {np.round(np.std(sreps),2)}")

In [ ]:
def calc_hom_over_personas(df, score='bert'):
    homs = []
    for persona_id in tqdm(df.persona_id.unique()):
        responses = df.loc[df.persona_id==persona_id].drop_duplicates(subset=['prompt_id'])['response'].values.tolist()
        
        hom = homogenization_score(responses, 'bertscore', verbose=False)
        homs.append(hom)
        
    print(f"Hom-{score}: {np.round(np.mean(homs),2)} ± {np.round(np.std(homs),2)}")

In [ ]:
print(homogenization_score(human_responses, 'bertscore', verbose=True))

# Llama-8B

## No persona, no cutoff

In [ ]:
np_df = pd.read_csv('../output/llama-temp0.7/llama8b-np/Llama-3.1-8B-Instruct_dolly_output.tsv', sep='\t')
np_df['response'] = np_df.response.apply(lambda x: x.strip())
np_df['prompt_id'] = [i for i in range(len(prompts))]
np_df['prompt'] = np_df.prompt_id.apply(lambda x: prompts[x])
np_df['persona_id'] = [-1 for i in range(len(prompts))]
np_df.head(1)

## Figure out the language of response

In [ ]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)

In [ ]:
np_df['lang'] = np_df.response.progress_apply(lambda x: model.predict(x.replace('\n', ' '))[0][0])

In [ ]:
Counter(np_df.lang.values.tolist())

In [ ]:
calc_cr_nds_over_personas(np_df)

## No persona with cutoff

In [ ]:
npc_df = pd.read_csv('../output/llama8b-cutoff-np/Llama-3.1-8B-Instruct_dolly_output.tsv', sep='\t')
npc_df['response'] = npc_df.response.apply(lambda x: x.strip())
npc_df['prompt_id'] = [i for i in range(len(prompts))]
npc_df['prompt'] = npc_df.prompt_id.apply(lambda x: prompts[x])
npc_df['persona_id'] = [-1 for i in range(len(prompts))]

npc_df.shape

In [ ]:
calc_cr_nds_over_personas(npc_df)

## Persona, no cutoff

In [ ]:
persona_df = pd.read_csv('../output/llama8b-persona/Llama-3.1-8B-Instruct_dolly_output.tsv', sep='\t')
persona_df['response'] = persona_df.response.apply(lambda x: x.strip())

persona_ids = []
prompt_ids = []
for persona_id in range(100):
    for prompt_id in range(100):
        persona_ids += [persona_id]
        prompt_ids += [prompt_id]
if 'persona_id' not in persona_df.columns:
    persona_df['prompt_id'] = prompt_ids
    persona_df['prompt'] = persona_df.prompt_id.apply(lambda x: prompts[x])
    persona_df['persona_id'] = persona_ids
    persona_df['persona'] = persona_df.persona_id.apply(lambda x: personas[x])

In [ ]:
calc_cr_nds_over_personas(persona_df)

### Now to check the lang of responses and get metrics with non-english removed

In [ ]:
persona_df['lang'] = persona_df.response.progress_apply(lambda x: model.predict(x.replace('\n', ' '))[0][0])

In [ ]:
for p in persona_df[persona_df.lang!='__label__eng_Latn']['persona_id'].unique():
    print(p,personas[p])

print(persona_df[persona_df.lang!='__label__eng_Latn']['persona_id'].unique().tolist())

In [ ]:
noneng_counts=Counter(persona_df[persona_df.lang!='__label__eng_Latn']['persona_id'].values.tolist())
print(noneng_counts)

In [ ]:
possible_noneng_personas = [x for x in noneng_counts if noneng_counts[x]>2]
print(possible_noneng_personas)

First just try all prompt responses labelled as english

In [ ]:
eng_persona_df = persona_df[persona_df.lang=='__label__eng_Latn']
print(eng_persona_df.shape)
calc_cr_nds_over_personas(eng_persona_df)

Now to try with all non-english personas possibly removed

In [ ]:
eng_persona_df2 = persona_df.loc[~persona_df.persona_id.isin(possible_noneng_personas)]
print(eng_persona_df2.shape)
calc_cr_nds_over_personas(eng_persona_df2)

## Persona plus cutoff

In [ ]:
personac_df = pd.read_csv('../output/llama8b-cutoff-persona/Llama-3.1-8B-Instruct_dolly_output.tsv', sep='\t')
personac_df['response'] = personac_df.response.apply(lambda x: x.strip())

persona_ids = []
prompt_ids = []
for persona_id in range(100):
    for prompt_id in range(100):
        persona_ids += [persona_id]
        prompt_ids += [prompt_id]
if 'persona_id' not in personac_df.columns:
    personac_df['prompt_id'] = prompt_ids
    personac_df['prompt'] = personac_df.prompt_id.apply(lambda x: prompts[x])
    personac_df['persona_id'] = persona_ids
    personac_df['persona'] = personac_df.persona_id.apply(lambda x: personas[x])

personac_df.head(3)

In [ ]:
calc_cr_nds_over_personas(personac_df)

### Now to check the lang of responses and get metrics with non-english removed

In [ ]:
personac_df['lang'] = personac_df.response.progress_apply(lambda x: model.predict(x.replace('\n', ' '))[0][0])

In [ ]:
for p in personac_df[personac_df.lang!='__label__eng_Latn']['persona_id'].unique():
    print(p,personas[p])

print(personac_df[personac_df.lang!='__label__eng_Latn']['persona_id'].unique().tolist())

In [ ]:
noneng_counts=Counter(personac_df[personac_df.lang!='__label__eng_Latn']['persona_id'].values.tolist())
print(noneng_counts)

In [ ]:
possible_noneng_personas = [x for x in noneng_counts if noneng_counts[x]>2]
print(possible_noneng_personas)

In [ ]:
personac_df[personac_df.lang!='__label__eng_Latn']['prompt_id'].unique().shape

First just try all prompt responses labelled as english

In [ ]:
eng_personac_df = personac_df[personac_df.lang=='__label__eng_Latn']
print(eng_personac_df.shape)
calc_cr_nds_over_personas(eng_personac_df)

Now to try with all possibly non-english personas removed

In [ ]:
eng_personac_df2 = personac_df.loc[~personac_df.persona_id.isin(possible_noneng_personas)]
print(eng_personac_df2.shape)
calc_cr_nds_over_personas(eng_personac_df2)

In [ ]:
personac_df[personac_df.lang!='__label__eng_Latn'].sample(5)

In [ ]:
personac_df.loc[((personac_df.persona_id==99) & (personac_df.lang!='__label__eng_Latn'))]

# Llama-70B temp=0.7

In [20]:
# No persona, no cutoff
np70_df = pd.read_csv('../output/llama-temp0.7/llama70b-np/Llama-3.3-70B-Instruct-Turbo_dolly_output.tsv', sep='\t')
np70_df['response'] = np70_df.response.apply(lambda x: x.strip())
np70_df['prompt_id'] = [i for i in range(len(prompts))]
np70_df['persona_id'] = [-1 for i in range(len(prompts))]
calc_cr_nds_over_personas(np70_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.03s/it]

CR: 2.76 ± 0.0
CR-POS: 5.72 ± 0.0
NDS: 2.87 ± 0.0
Self-rep:1.96 ± 0.0


In [21]:
# No persona with cutoff
npc70_df = pd.read_csv('../output/llama-temp0.7/llama70b-cutoff-np/Llama-3.3-70B-Instruct-Turbo_dolly_output.tsv', sep='\t')
npc70_df['response'] = npc70_df.response.apply(lambda x: x.strip())
npc70_df['prompt_id'] = [i for i in range(len(prompts))]
npc70_df['persona_id'] = [-1 for i in range(len(prompts))]
calc_cr_nds_over_personas(npc70_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.21s/it]

CR: 2.57 ± 0.0
CR-POS: 5.1 ± 0.0
NDS: 3.1 ± 0.0
Self-rep:0.44 ± 0.0


In [9]:
# Persona plus cutoff
personac70_df = pd.read_csv('../output/llama-temp0.7/llama70b-cutoff-persona/Llama-3.3-70B-Instruct-Turbo_dolly_output.tsv', sep='\t')
personac70_df['response'] = personac70_df.response.apply(lambda x: x.strip())

if 'persona_id' not in personac70_df.columns:
    print("Adding persona prompt ids")
    persona_ids = []
    prompt_ids = []
    for persona_id in range(100):
        for prompt_id in range(100):
            persona_ids += [persona_id]
            prompt_ids += [prompt_id]
    personac70_df['prompt_id'] = prompt_ids
    personac70_df['prompt'] = personac70_df.prompt_id.apply(lambda x: prompts[x])
    personac70_df['persona_id'] = persona_ids
    personac70_df['persona'] = personac70_df.persona_id.apply(lambda x: personas[x])

personac70_df.head(3)

,prompt_id,persona_id,response
0,0,0,"As a PR manager, I'd frame your request as a compelling narrative. Emphasize your academic goals and how increased financial support will enable you to focus on your studies, leading to better grades and a brighter future. Highlight specific expenses, such as textbooks, transportation, or extracurricular activities, that the current amount doesn't cover. \n\nYou could say, ""Mom, I understand that you want me to be responsible with money, but I believe that with a bit more support, I can achieve my academic goals and make you proud. I've done some research and found that my expenses are higher than what I currently receive. I'd like to discuss a possible increase to ensure I can cover all my necessary expenses and focus on my studies without added stress."" This approach shows that you've thoughtfully considered your needs and are committed to your education."
1,1,0,"I understand where my colleague is coming from, but as a PR manager, I've seen how misinformation can spread like wildfire. The notion that ancient structures like Stonehenge and the Great Pyramid are evidence of UFO activities is a fascinating narrative, but it's essential to separate fact from fiction. These iconic landmarks have been extensively studied by archaeologists, historians, and scientists, who attribute their construction to the ingenuity and skill of our ancient ancestors. While it's intriguing to consider alternative explanations, there's no credible evidence to support the claim of UFO involvement. Let's focus on promoting accurate information and respecting the cultural heritage of these incredible achievements."
2,2,0,"As a PR manager, I advise influencers to use the STAR method: Situation, Task, Action, Result. This framework helps them structure their response, providing context, highlighting their actions, and showcasing the outcome, making their answer clear, concise, and engaging for the audience."


In [10]:
calc_cr_nds_over_personas(personac70_df)

  2%|█▉                                                                                             | 2/100 [00:13<10:39,  6.52s/it]
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/venkat/micromamba/envs/diversity/lib/python3.11/multiprocessing/spawn.py", line 120, in spawn_main
Process Process-9:
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/venkat/micromamba/envs/diversity/lib/python3.11/multiprocessing/spawn.py", line 130, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/venkat/micromamba/envs/diversity/lib/python3.11/site-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/Users/venkat/micromamba/envs/diversity/lib/python3.11/site-packages/spacy/errors.py", line 3, in <module>
    from .compat import Literal
  File "/Users/venkat/micromamba/envs/diversity/lib/python3.11/s

KeyboardInterrupt: 

In [ ]:
personac70_df['lang'] = personac70_df.response.progress_apply(lambda x: model.predict(x.replace('\n', ' '))[0][0])

In [ ]:
for p in personac70_df[personac70_df.lang!='__label__eng_Latn']['persona_id'].unique():
    print(p,personas[p])

print(personac70_df[personac70_df.lang!='__label__eng_Latn']['persona_id'].unique().tolist())

In [ ]:
noneng_counts=Counter(personac70_df[personac70_df.lang!='__label__eng_Latn']['persona_id'].values.tolist())
print(noneng_counts)

In [ ]:
possible_noneng_personas = [x for x in noneng_counts if noneng_counts[x]>2]
print(possible_noneng_personas)

In [ ]:
personac70_df[personac70_df.lang!='__label__eng_Latn']['prompt_id'].unique().shape

First just try all prompt responses labelled as english

In [ ]:
eng_personac70_df = personac70_df[personac70_df.lang=='__label__eng_Latn']
print(eng_personac70_df.shape)
calc_cr_nds_over_personas(eng_personac70_df)

Now to try with all possibly non-english personas removed

In [ ]:
eng_personac70_df2 = personac70_df.loc[~personac70_df.persona_id.isin(possible_noneng_personas)]
print(eng_personac70_df2.shape)
calc_cr_nds_over_personas(eng_personac70_df2)

In [22]:
# Persona no cutoff
persona70_df = pd.read_csv('../output/llama-temp0.7/llama70b-persona/Llama-3.3-70B-Instruct-Turbo_dolly_output.tsv', sep='\t')
persona70_df['response'] = persona70_df.response.apply(lambda x: x.strip())

if 'persona_id' not in persona70_df.columns:
    print("Adding persona prompt ids")
    persona_ids = []
    prompt_ids = []
    for persona_id in range(100):
        for prompt_id in range(100):
            persona_ids += [persona_id]
            prompt_ids += [prompt_id]
    persona70_df['prompt_id'] = prompt_ids
    persona70_df['prompt'] = persona70_df.prompt_id.apply(lambda x: prompts[x])
    persona70_df['persona_id'] = persona_ids
    persona70_df['persona'] = persona70_df.persona_id.apply(lambda x: personas[x])

persona70_df = persona70_df.drop_duplicates(subset=['prompt_id', 'persona_id'], keep='first')
persona70_df.shape

(10000, 3)

In [23]:
calc_cr_nds_over_personas(persona70_df)

Process Process-104:                                                                                        | 0/100 [00:00<?, ?it/s]
Process Process-101:
Process Process-102:
Process Process-103:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/venkat/micromamba/envs/diversity/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/venkat/micromamba/envs/diversity/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/venkat/micromamba/envs/diversity/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/venkat/micromamba/envs/diversity/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/venkat/micromamba/envs/diversity/lib/python3.11/site-packages/spacy/language.

KeyboardInterrupt: 

In [24]:
crs = []
ndss = []
crs_pos = []
sreps = []
newpc = persona70_df.set_index(['persona_id', 'prompt_id'])
for _ in tqdm(range(10)):
    # Get random personas paired with every prompt
    persona_ids_shuffled = [i for i in range(100)]
    shuffle(persona_ids_shuffled)
    prompt_ids = [i for i in range(100)]
    pairs = list(zip(persona_ids_shuffled, prompt_ids))
    responses = newpc.loc[pairs, 'response'].values.tolist()
    
    # Calculate metrics
    cr, cr_pos, nds, srep = calc_cr_nds_sr(responses)

    crs.append(cr)
    ndss.append(nds)
    crs_pos.append(cr_pos)
    sreps.append(srep)

print(f"CR: {np.round(np.mean(crs),2)} ± {np.round(np.std(crs),2)}\nCR-POS: {np.round(np.mean(crs_pos),2)} ± {np.round(np.std(crs_pos),2)}\nNDS: {np.round(np.mean(ndss),2)} ± {np.round(np.std(ndss),2)}\nSelf-rep:{np.round(np.mean(sreps),2)} ± {np.round(np.std(sreps),2)}")

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:00<00:00,  6.08s/it]

CR: 2.7 ± 0.01
CR-POS: 5.37 ± 0.03
NDS: 2.84 ± 0.01
Self-rep:2.53 ± 0.11


In [ ]:
calc_hom_over_personas(persona70_df)

In [ ]:
persona70_df['lang'] = persona70_df.response.progress_apply(lambda x: model.predict(x.replace('\n', ' '))[0][0])

In [ ]:
for p in persona70_df[persona70_df.lang!='__label__eng_Latn']['persona_id'].unique():
    print(p,personas[p])

print(persona70_df[persona70_df.lang!='__label__eng_Latn']['persona_id'].unique().tolist())

In [ ]:
noneng_counts=Counter(persona70_df[persona70_df.lang!='__label__eng_Latn']['persona_id'].values.tolist())
print(noneng_counts)

In [ ]:
possible_noneng_personas = [x for x in noneng_counts if noneng_counts[x]>1]
print(possible_noneng_personas)

In [ ]:
persona70_df[persona70_df.lang!='__label__eng_Latn']['prompt_id'].unique().shape

First just try all prompt responses labelled as english

In [ ]:
eng_persona70_df = persona70_df[persona70_df.lang=='__label__eng_Latn']
print(eng_persona70_df.shape)
calc_cr_nds_over_personas(eng_persona70_df)

Now to try with all possibly non-english personas removed

In [ ]:
eng_persona70_df2 = persona70_df.loc[~persona70_df.persona_id.isin(possible_noneng_personas)]
print(eng_persona70_df2.shape)
calc_cr_nds_over_personas(eng_persona70_df2)

In [13]:
# What if every prompt was answered by a different personas?
def calc_cr_nds_sr(responses):
    cr = compression_ratio(responses, 'gzip')
    nds = ngram_diversity_score(responses, 4)
    #CR-POS
    joined_pos, tuples = get_pos(responses)
    # ngrams_pos = token_patterns(joined_pos, 5, 10)
    cr_pos = compression_ratio(joined_pos, 'gzip')
    srep = self_repetition_score(responses, verbose=False)
    return cr, cr_pos, nds, srep

In [15]:
crs = []
ndss = []
crs_pos = []
sreps = []
newpc = personac70_df.set_index(['persona_id', 'prompt_id'])
for _ in tqdm(range(10)):
    # Get random personas paired with every prompt
    persona_ids_shuffled = [i for i in range(100)]
    shuffle(persona_ids_shuffled)
    prompt_ids = [i for i in range(100)]
    pairs = list(zip(persona_ids_shuffled, prompt_ids))
    responses = newpc.loc[pairs, 'response'].values.tolist()
    
    # Calculate metrics
    cr, cr_pos, nds, srep = calc_cr_nds_sr(responses)

    crs.append(cr)
    ndss.append(nds)
    crs_pos.append(cr_pos)
    sreps.append(srep)

print(f"CR: {np.round(np.mean(crs),2)} ± {np.round(np.std(crs),2)}\nCR-POS: {np.round(np.mean(crs_pos),2)} ± {np.round(np.std(crs_pos),2)}\nNDS: {np.round(np.mean(ndss),2)} ± {np.round(np.std(ndss),2)}\nSelf-rep:{np.round(np.mean(sreps),2)} ± {np.round(np.std(sreps),2)}")

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:54<00:00,  5.49s/it]

CR: 2.51 ± 0.02
CR-POS: 5.04 ± 0.03
NDS: 3.08 ± 0.02
Self-rep:0.69 ± 0.14


# Llama-70b temp=1

In [9]:
# No persona, no cutoff
np70_df = pd.read_csv('../output/llama-temp1/llama70b-np/Llama-3.3-70B-Instruct-Turbo_dolly_output.tsv', sep='\t')
np70_df['response'] = np70_df.response.apply(lambda x: x.strip())
np70_df['prompt_id'] = [i for i in range(len(prompts))]
np70_df['persona_id'] = [-1 for i in range(len(prompts))]
calc_cr_nds_over_personas(np70_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.62s/it]

CR: 2.77 ± 0.0
CR-POS: 5.65 ± 0.0
NDS: 2.87 ± 0.0
Self-rep:5.79 ± 0.0


In [10]:
# No persona with cutoff
npc70_df = pd.read_csv('../output/llama-temp1/llama70b-cutoff-np/Llama-3.3-70B-Instruct-Turbo_dolly_output.tsv', sep='\t')
npc70_df['response'] = npc70_df.response.apply(lambda x: x.strip())
npc70_df['prompt_id'] = [i for i in range(len(prompts))]
npc70_df['persona_id'] = [-1 for i in range(len(prompts))]
calc_cr_nds_over_personas(npc70_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.76s/it]

CR: 2.57 ± 0.0
CR-POS: 5.13 ± 0.0
NDS: 3.08 ± 0.0
Self-rep:4.36 ± 0.0


In [12]:
# Persona plus cutoff
personac70_df = pd.read_csv('../output/llama-temp1/llama70b-cutoff-persona/Llama-3.3-70B-Instruct-Turbo_dolly_output.tsv', sep='\t')
personac70_df['response'] = personac70_df.response.apply(lambda x: x.strip())

if 'persona_id' not in personac70_df.columns:
    print("Adding persona prompt ids")
    persona_ids = []
    prompt_ids = []
    for persona_id in range(100):
        for prompt_id in range(100):
            persona_ids += [persona_id]
            prompt_ids += [prompt_id]
    personac70_df['prompt_id'] = prompt_ids
    personac70_df['prompt'] = personac70_df.prompt_id.apply(lambda x: prompts[x])
    personac70_df['persona_id'] = persona_ids
    personac70_df['persona'] = personac70_df.persona_id.apply(lambda x: personas[x])

calc_cr_nds_over_personas(personac70_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:00<00:00,  3.60s/it]

CR: 2.74 ± 0.1
CR-POS: 5.22 ± 0.16
NDS: 2.88 ± 0.08
Self-rep:4.55 ± 0.16


# Deepseek

In [13]:
# No persona, no cutoff
deep_np_df = pd.read_csv('../output/deepseek-np/DeepSeek-V3_dolly_output.tsv', sep='\t')
deep_np_df['response'] = deep_np_df.response.apply(lambda x: x.strip())
deep_np_df['prompt_id'] = [i for i in range(len(prompts))]
deep_np_df['persona_id'] = [-1 for i in range(len(prompts))]
calc_cr_nds_over_personas(deep_np_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.52s/it]

CR: 2.57 ± 0.0
CR-POS: 5.56 ± 0.0
NDS: 3.02 ± 0.0
Self-rep:5.75 ± 0.0


In [15]:
# No persona, plus cutoff
deep_npc_df = pd.read_csv('../output/deepseek-np-cutoff/DeepSeek-V3_dolly_output.tsv', sep='\t')
deep_npc_df['response'] = deep_npc_df.response.apply(lambda x: x.strip())
deep_npc_df['prompt_id'] = [i for i in range(len(prompts))]
deep_npc_df['persona_id'] = [-1 for i in range(len(prompts))]
calc_cr_nds_over_personas(deep_npc_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.55s/it]

CR: 2.45 ± 0.0
CR-POS: 5.04 ± 0.0
NDS: 3.22 ± 0.0
Self-rep:4.81 ± 0.0


# Response length distribution

In [ ]:
len_df = {'len': [], 'source':[]}

# Load all the human responses first.
len_df['len'] += sample['response'].apply(lambda x: len(x)).values.tolist()
len_df['source'] += ['dolly' for _ in range(len(sample))]

for (df, source_name) in [(np_df, 'No persona'), (npc_df,'No persona+cutoff'), (persona_df, 'Persona'), (personac_df, 'Persona+cutoff')]:
    len_df['len'] += df['response'].apply(lambda x: len(x)).values.tolist()
    len_df['source'] += [source_name for _ in range(len(df))]
    
len_df=pd.DataFrame(len_df)

g = sns.kdeplot(len_df, x='len', hue='source', common_norm=False, fill=True, clip=[-1000,6000])
g.set_xlabel('Completion length (chars)')
g.set_ylabel('Density')
g.spines["top"].set_visible(False)
g.spines["right"].set_visible(False)
g.spines["left"].set_visible(False)
g.set(yticklabels=[])
g.grid(axis='x')
# plt.legend([], [], frameon=False)
plt.show()

# llama-1b

In [ ]:
np1_df = pd.read_csv('../output/llama1b-np/Llama-3.2-1B-Instruct_dolly_output.tsv', sep='\t')
np1_df['response'] = np1_df.response.apply(lambda x: x.strip())
np1_df['prompt_id'] = [i for i in range(len(prompts))]
np1_df['persona_id'] = [-1 for i in range(len(prompts))]
calc_cr_nds_over_personas(np1_df)

In [ ]:
npc1_df = pd.read_csv('../output/llama1b-cutoff-np/Llama-3.2-1B-Instruct_dolly_output.tsv', sep='\t')
npc1_df['response'] = npc1_df.response.apply(lambda x: x.strip())
npc1_df['prompt_id'] = [i for i in range(len(prompts))]
npc1_df['persona_id'] = [-1 for i in range(len(prompts))]
calc_cr_nds_over_personas(npc1_df)

In [ ]:
persona1_df = pd.read_csv('../output/llama1b-persona/Llama-3.2-1B-Instruct_dolly_output.tsv', sep='\t')
persona1_df['response'] = persona1_df.response.apply(lambda x: x.strip())

persona_ids = []
prompt_ids = []
for persona_id in range(100):
    for prompt_id in range(100):
        persona_ids += [persona_id]
        prompt_ids += [prompt_id]
persona1_df['prompt_id'] = prompt_ids
persona1_df['persona_id'] = persona_ids
calc_cr_nds_over_personas(persona1_df)

In [ ]:
persona1_df['lang'] = persona1_df.response.progress_apply(lambda x: model.predict(x.replace('\n', ' '))[0][0])

In [ ]:
for p in persona1_df[persona1_df.lang!='__label__eng_Latn']['persona_id'].unique():
    print(p,personas[p])

print(persona1_df[persona1_df.lang!='__label__eng_Latn']['persona_id'].unique().tolist())

In [ ]:
noneng_counts=Counter(persona1_df[persona1_df.lang!='__label__eng_Latn']['persona_id'].values.tolist())
print(noneng_counts)

In [ ]:
possible_noneng_personas = [x for x in noneng_counts if noneng_counts[x]>2]
print(possible_noneng_personas)

In [ ]:
persona1_df[persona1_df.lang!='__label__eng_Latn']['prompt_id'].unique().shape

First just try all prompt responses labelled as english

In [ ]:
eng_persona1_df = persona1_df[persona1_df.lang=='__label__eng_Latn']
print(eng_persona1_df.shape)
calc_cr_nds_over_personas(eng_persona1_df)

Now to try with all possibly non-english personas removed

In [ ]:
eng_persona1_df2 = persona1_df.loc[~persona1_df.persona_id.isin(possible_noneng_personas)]
print(eng_persona1_df2.shape)
calc_cr_nds_over_personas(eng_persona1_df2)

In [ ]:
# Persona plus cut off
personac1_df = pd.read_csv('../output/llama1b-cutoff-persona/Llama-3.2-1B-Instruct_dolly_output.tsv', sep='\t')
personac1_df['response'] = personac1_df.response.apply(lambda x: x.strip())

persona_ids = []
prompt_ids = []
for persona_id in range(100):
    for prompt_id in range(100):
        persona_ids += [persona_id]
        prompt_ids += [prompt_id]
personac1_df['prompt_id'] = prompt_ids
personac1_df['persona_id'] = persona_ids

calc_cr_nds_over_personas(personac1_df)

In [ ]:
personac1_df['lang'] = personac1_df.response.progress_apply(lambda x: model.predict(x.replace('\n', ' '))[0][0])

In [ ]:
for p in personac1_df[personac1_df.lang!='__label__eng_Latn']['persona_id'].unique():
    print(p,personas[p])

print(personac1_df[personac1_df.lang!='__label__eng_Latn']['persona_id'].unique().tolist())

In [ ]:
noneng_counts=Counter(personac1_df[personac1_df.lang!='__label__eng_Latn']['persona_id'].values.tolist())
print(noneng_counts)

In [ ]:
possible_noneng_personas = [x for x in noneng_counts if noneng_counts[x]>2]
print(possible_noneng_personas)

In [ ]:
personac1_df[personac1_df.lang!='__label__eng_Latn']['prompt_id'].unique().shape

First just try all prompt responses labelled as english

In [ ]:
eng_personac1_df = personac1_df[personac1_df.lang=='__label__eng_Latn']
print(eng_personac1_df.shape)
calc_cr_nds_over_personas(eng_personac1_df)

Now to try with all possibly non-english personas removed

In [ ]:
eng_personac1_df2 = personac1_df.loc[~personac1_df.persona_id.isin(possible_noneng_personas)]
print(eng_personac1_df2.shape)
calc_cr_nds_over_personas(eng_personac1_df2)

## Llama70B coarse

In [17]:
# Persona plus cutoff plus coarse personas
persona70_df_coarse = pd.read_csv('../output/coarse/llama-cutoff-persona/Llama-3.3-70B-Instruct-Turbo_dolly_output.tsv', sep='\t')
persona70_df_coarse['response'] = persona70_df_coarse.response.apply(lambda x: x.strip())

persona70_df_coarse = persona70_df_coarse.drop_duplicates(subset=['prompt_id', 'persona_id'], keep='first')
persona70_df_coarse.shape

(10000, 3)

In [19]:
crs = []
ndss = []
crs_pos = []
sreps = []
newpc = persona70_df_coarse.set_index(['persona_id', 'prompt_id'])
for _ in tqdm(range(10)):
    # Get random personas paired with every prompt
    persona_ids_shuffled = [i for i in range(100)]
    shuffle(persona_ids_shuffled)
    prompt_ids = [i for i in range(100)]
    pairs = list(zip(persona_ids_shuffled, prompt_ids))
    responses = newpc.loc[pairs, 'response'].values.tolist()
    
    # Calculate metrics
    cr, cr_pos, nds, srep = calc_cr_nds_sr(responses)

    crs.append(cr)
    ndss.append(nds)
    crs_pos.append(cr_pos)
    sreps.append(srep)

print(f"CR: {np.round(np.mean(crs),2)} ± {np.round(np.std(crs),2)}\nCR-POS: {np.round(np.mean(crs_pos),2)} ± {np.round(np.std(crs_pos),2)}\nNDS: {np.round(np.mean(ndss),2)} ± {np.round(np.std(ndss),2)}\nSelf-rep:{np.round(np.mean(sreps),2)} ± {np.round(np.std(sreps),2)}")

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:59<00:00,  5.93s/it]

CR: 2.52 ± 0.02
CR-POS: 5.07 ± 0.03
NDS: 3.08 ± 0.02
Self-rep:0.62 ± 0.07
